In [1]:
import requests

In [2]:
url = "https://api.nasa.gov/neo/rest/v1/neo/browse?page=1&api_key=RxmRVoLLKKiUuuKLTytCZD7qBlGayWQ5NeeeRCZ6"

In [3]:
resp = requests.get(url)

In [4]:
data = resp.json()

In [5]:
data['page']

{'size': 20, 'total_elements': 25981, 'total_pages': 1300, 'number': 1}